# Lab 1: Redis Environment & CLI Basics

**Duration:** 45 minutes  
**Objective:** Master Redis environment setup and CLI operations using Python

## 🎯 Learning Objectives

- Set up Docker Redis environment
- Connect to Redis using Python redis-py library
- Execute basic Redis operations with Python
- Monitor Redis operations programmatically
- Implement error handling and connection management

## Part 1: Environment Setup (10 minutes)

In [ ]:
# Install required packages
!pip install redis docker python-dotenv tabulate colorama pandas

In [ ]:
import redis
import docker
import json
import time
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv
from tabulate import tabulate
from colorama import init, Fore, Style
import pandas as pd

# Initialize colorama for colored output
init(autoreset=True)

# Load environment variables
load_dotenv()

print(f"{Fore.GREEN}✅ Libraries imported successfully{Style.RESET_ALL}")

### Step 1: Start Redis Container

In [ ]:
# Docker client setup
docker_client = docker.from_env()

# Check if Redis container exists and remove if it does
try:
    existing_container = docker_client.containers.get('redis-lab1')
    existing_container.stop()
    existing_container.remove()
    print(f"{Fore.YELLOW}⚠️ Removed existing container{Style.RESET_ALL}")
except docker.errors.NotFound:
    pass

# Start new Redis container
container = docker_client.containers.run(
    'redis:7-alpine',
    name='redis-lab1',
    ports={'6379/tcp': 6379},
    detach=True,
    command='redis-server --maxmemory 256mb --maxmemory-policy allkeys-lru'
)

# Wait for container to be ready
time.sleep(2)
print(f"{Fore.GREEN}✅ Redis container 'redis-lab1' started successfully{Style.RESET_ALL}")
print(f"Container ID: {container.short_id}")

### Step 2: Connect to Redis

In [ ]:
# Create Redis connection
try:
    r = redis.Redis(
        host='localhost',
        port=6379,
        db=0,
        decode_responses=True  # Decode responses to strings
    )
    
    # Test connection
    ping_response = r.ping()
    print(f"{Fore.GREEN}✅ Connected to Redis: {ping_response}{Style.RESET_ALL}")
    
    # Get server info
    info = r.info('server')
    print(f"\nRedis Version: {info['redis_version']}")
    print(f"Mode: {info['redis_mode']}")
    print(f"TCP Port: {info['tcp_port']}")
    
except redis.ConnectionError as e:
    print(f"{Fore.RED}❌ Failed to connect to Redis: {e}{Style.RESET_ALL}")

## Part 2: Basic Redis Operations (15 minutes)

### Working with Strings

In [ ]:
# String operations
print(f"{Fore.CYAN}=== STRING OPERATIONS ==={Style.RESET_ALL}\n")

# SET and GET
r.set('customer:1001:name', 'John Smith')
r.set('customer:1001:email', 'john.smith@example.com')
r.set('customer:1001:policy', 'POL-2024-1001')

# Retrieve values
name = r.get('customer:1001:name')
email = r.get('customer:1001:email')
policy = r.get('customer:1001:policy')

print(f"Customer Name: {name}")
print(f"Email: {email}")
print(f"Policy: {policy}")

# SET with expiration (TTL)
r.setex('session:abc123', 3600, 'user:1001')  # Expires in 1 hour
ttl = r.ttl('session:abc123')
print(f"\nSession TTL: {ttl} seconds")

# Atomic increment/decrement
r.set('quote:counter', '1000')
new_quote_id = r.incr('quote:counter')
print(f"\nNew Quote ID: QUO-{new_quote_id}")

### Working with Multiple Keys

In [ ]:
# Batch operations with MSET and MGET
print(f"{Fore.CYAN}=== BATCH OPERATIONS ==={Style.RESET_ALL}\n")

# Set multiple keys at once
policy_data = {
    'policy:POL-2024-1001:type': 'AUTO',
    'policy:POL-2024-1001:premium': '1200',
    'policy:POL-2024-1001:status': 'ACTIVE',
    'policy:POL-2024-1001:start_date': '2024-01-01',
    'policy:POL-2024-1001:end_date': '2025-01-01'
}

r.mset(policy_data)
print("✅ Policy data saved")

# Get multiple keys at once
keys = list(policy_data.keys())
values = r.mget(keys)

# Display as table
table_data = []
for key, value in zip(keys, values):
    field = key.split(':')[-1]
    table_data.append([field, value])

print("\nPolicy Details:")
print(tabulate(table_data, headers=['Field', 'Value'], tablefmt='grid'))

## Part 3: Redis Monitoring (10 minutes)

In [ ]:
# Create a monitoring function
def monitor_redis_operations(duration=5):
    """Monitor Redis operations for specified duration."""
    print(f"{Fore.CYAN}=== MONITORING REDIS OPERATIONS ==={Style.RESET_ALL}")
    print(f"Monitoring for {duration} seconds...\n")
    
    # Create a separate connection for monitoring
    monitor_conn = redis.Redis(host='localhost', port=6379, decode_responses=True)
    
    # Get initial stats
    info_before = monitor_conn.info('stats')
    ops_before = info_before.get('total_commands_processed', 0)
    
    # Perform some operations
    for i in range(10):
        r.set(f'test:key:{i}', f'value_{i}')
        r.get(f'test:key:{i}')
        time.sleep(0.5)
    
    # Get final stats
    info_after = monitor_conn.info('stats')
    ops_after = info_after.get('total_commands_processed', 0)
    
    # Calculate and display statistics
    ops_executed = ops_after - ops_before
    print(f"\n📊 Statistics:")
    print(f"  Total operations executed: {ops_executed}")
    print(f"  Operations per second: {ops_executed/duration:.2f}")
    print(f"  Total connections received: {info_after.get('total_connections_received', 0)}")
    print(f"  Instantaneous ops/sec: {info_after.get('instantaneous_ops_per_sec', 0)}")

# Run monitoring
monitor_redis_operations(5)

## Part 4: Cleanup

In [ ]:
# Cleanup function
def cleanup():
    """Clean up Redis keys and close connections."""
    # Delete test keys
    for key in r.keys('test:*'):
        r.delete(key)
    
    print(f"{Fore.GREEN}✅ Cleanup completed{Style.RESET_ALL}")

cleanup()